In [1]:
cd("G:\\My Drive\\_MOPJ\\Julia\\ROM4SM\\")

In [2]:
readdir()

6-element Array{String,1}:
 ".git"
 "LICENSE"
 "POD4Frame.ipynb"
 "POD4Frame.jl"
 "POD4Vectors.jl"
 "img"

# Treillis
<img src="https://raw.githubusercontent.com/amdeld/ROM4SM.jl/master/img/TrussStructure_LBC.png" alt="Drawing" style="width: 1000px;"/>


# Fonction Matrice de raideur élémentaire pour un élément de barre plan

In [3]:
function trstiff(coo,E,A)
# Function that computes stiffness matrix for truss element
# (file name trstiff.m)
L=sqrt((coo[2,1]-coo[1,1])^2+(coo[2,2]-coo[1,2])^2)
C=(coo[2,1]-coo[1,1])/L
S=(coo[2,2]-coo[1,2])/L
# Stiffness matrix
STF=E*A/L*[C^2 S*C -C^2 -S*C;
S*C S^2 -S*C -S^2;
-C^2 -S*C C^2 S*C;
-S*C -S^2 S*C S^2];
return(STF)
end

trstiff (generic function with 1 method)

# Fonction d'assemblage

In [4]:
# Main program for truss structure
# Coordinates - units L[mm]
function truss_str(E,A)
COO=[0 0; 2 0; 4 0 ;1 1; 3 1; -1 1; 5 1]
COO=1000*COO
DOF=size(COO,1)*2
# Elements
ELM=[1 2; 2 3; 6 4; 4 5; 5 7; 1 6; 1 4; 2 4; 2 5; 3 5; 3 7]
# Properties (units F[N], L[mm])
#E=125000
#A=6
# Assembling of stiffness matrix
MSTF=zeros(DOF,DOF)
X=zeros(2,2)
for i in 1:size(ELM,1)
X[1,:]=COO[ELM[i,1],:]
X[2,:]=COO[ELM[i,2],:]
STF=trstiff(X,E,A)
MSTF[2*ELM[i,1]-1:2*ELM[i,1],2*ELM[i,1]-
1:2*ELM[i,1]]=MSTF[2*ELM[i,1]-1:2*ELM[i,1],2*ELM[i,1]-
1:2*ELM[i,1]]+STF[1:2,1:2]
MSTF[2*ELM[i,1]-1:2*ELM[i,1],2*ELM[i,2]-
1:2*ELM[i,2]]=MSTF[2*ELM[i,1]-1:2*ELM[i,1],2*ELM[i,2]-
1:2*ELM[i,2]]+STF[3:4,1:2]
MSTF[2*ELM[i,2]-1:2*ELM[i,2],2*ELM[i,1]-
1:2*ELM[i,1]]=MSTF[2*ELM[i,2]-1:2*ELM[i,2],2*ELM[i,1]-
1:2*ELM[i,1]]+STF[1:2,3:4]
MSTF[2*ELM[i,2]-1:2*ELM[i,2],2*ELM[i,2]-
1:2*ELM[i,2]]=MSTF[2*ELM[i,2]-1:2*ELM[i,2],2*ELM[i,2]-
1:2*ELM[i,2]]+STF[3:4,3:4]
end
# Constrains
con=[12 13 14 15] # constrained DOF
CDOF=size(con,2)
# Forces
dF=zeros(DOF,1)
dF[2]=-5000
dF[6]=-2000
# Reduced stiffness matrix & force vector
RSTF=MSTF[1:DOF-CDOF,1:DOF-CDOF]
dFR=dF[1:DOF-CDOF]
# Solving for displacements
d=inv(RSTF)*dFR
# End of main program
return(d)
end

truss_str (generic function with 1 method)

# Fonction Plan d'expériences (Design Of Experiments doe)

In [5]:
function doe()
    cnt = 1
    U=zeros(10)
    for E in [125000 150000 175000 200000]
        for A in [6 8 10 12]
            Disp = truss_str(E, A)
            #show(Disp)
            if cnt > 1
                U = [U Disp]
            else
                U = Disp
            end
            cnt += 1
        end
    end
    #show(U)
    return (U)
end

# Exécution du plan d'expériences
Variation du module de Young E = {125000; 150000; 175000; 200000} (unité MPa)

In [6]:
# Execute The design of experiments (DOE)
U=doe()
println("\nThe Data Matrix U coming out from the DOE\n")
U
#


The Data Matrix U coming out from the DOE



10×16 Array{Float64,2}:
 -11.5556     -8.66667    -6.93333   …   -5.41667    -4.33333    -3.61111
 -28.5261    -21.3946    -17.1157       -13.3716    -10.6973     -8.91441
  -0.888889   -0.666667   -0.533333      -0.416667   -0.333333   -0.277778
 -31.866     -23.8995    -19.1196       -14.9372    -11.9497     -9.95812
   7.11111     5.33333     4.26667        3.33333     2.66667     2.22222
 -16.5392    -12.4044     -9.92352   …   -7.75275    -6.2022     -5.1685
  -2.66667    -2.0        -1.6           -1.25       -1.0        -0.833333
 -35.5294    -26.647     -21.3176       -16.6544    -13.3235    -11.1029
  -2.66667    -2.0        -1.6           -1.25       -1.0        -0.833333
 -28.2026    -21.1519    -16.9216       -13.22      -10.576      -8.81331

In [ ]:
# Execute The design of experiments (DOE)
U=doe()
println("\nThe Data Matrix U coming out from the DOE\n")
display(U)
#

In [11]:
show("text/plain", U)

LoadError: MethodError: no method matching show(::String, ::Array{Float64,2})
Closest candidates are:
  show(!Matched::IO, ::AbstractArray) at arrayshow.jl:430
  show(!Matched::IO, ::Any) at show.jl:384
  show(::Any) at show.jl:386